In [ ]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import gc
import random
import numpy as np
import pandas as pd
from scipy.fft import fft
from scipy.signal import hilbert, blackman
import matplotlib.pyplot as plt
from IPython.display import display

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau


from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

%matplotlib inline

In [ ]:
class config:
    paths = {
        # data
        'train': '../input/ventilator-pressure-prediction/train.csv',
        'test' : '../input/ventilator-pressure-prediction/test.csv',
        'ss'   : '../input/ventilator-pressure-prediction/sample_submission.csv',    
    }
    
    model_params = {
        'debug'   : False,
        
        'EPOCH'   : 300,
        'BATCH_SIZE' : 512,
        'NUM_FOLDS' : 10,
    }
    
    post_processing = {
        'max_pressure': 64.82099173863948,
        'min_pressure': -1.8957442945646408,
        'diff_pressure': 0.07030215,
    }
    
    random_state = 1351

In [ ]:
def seed_all(seed=config.random_state):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_all()

In [ ]:
train = pd.read_csv(config.paths["train"])
test = pd.read_csv(config.paths["test"])
submission = pd.read_csv(config.paths["ss"])

if config.model_params["debug"]:
    print("[INFO] Debug Mode...")
    train = train[:80*1000]
    config.model_params["EPOCH"] = 2
    config.model_params["BATCH_SIZE"] = 128
    config.model_params["NUM_FOLDS"] = 5

In [ ]:
w = blackman(81)

ffta = lambda x: np.abs(fft(np.append(x.values,x.values[0]))[:80])
ffta.__name__ = 'ffta'

fftw = lambda x: np.abs(fft(np.append(x.values,x.values[0])*w)[:80])
fftw.__name__ = 'fftw'

def add_features(df):
    # new features
    df["cross_u_in"] = df["u_in"] * (1 - df["u_out"])
    df["cross_time_step"] = df["time_step"] * (1 - df["u_out"])
    df["area_frac"] = df["u_in"] * df["time_step"]
    df["cross_area_frac"] = df["area_frac"] * (1 - df["u_out"])

    # group by breath_id cache
    bid_cache = df.groupby(["breath_id"])
    
    # cumsum/diff features per breath id
    df["area_cumsum"] = bid_cache["area_frac"].cumsum()
    df["u_in_cumsum"] = bid_cache["u_in"].cumsum()
    df["timestep_cumsum"] = bid_cache["time_step"].cumsum()
    
    df['u_in_lag1'] = bid_cache['u_in'].shift(1)
    df['u_out_lag1'] = bid_cache['u_out'].shift(1)
    df['u_in_lag_back1'] = bid_cache['u_in'].shift(-1)
    df['u_out_lag_back1'] = bid_cache['u_out'].shift(-1)
    df['u_in_lag2'] = bid_cache['u_in'].shift(2)
    df['u_out_lag2'] = bid_cache['u_out'].shift(2)
    df['u_in_lag_back2'] = bid_cache['u_in'].shift(-2)
    df['u_out_lag_back2'] = bid_cache['u_out'].shift(-2)
    df['u_in_lag3'] = bid_cache['u_in'].shift(3)
    df['u_out_lag3'] = bid_cache['u_out'].shift(3)
    df['u_in_lag_back3'] = bid_cache['u_in'].shift(-3)
    df['u_out_lag_back3'] = bid_cache['u_out'].shift(-3)
    df['u_in_lag4'] = bid_cache['u_in'].shift(4)
    df['u_out_lag4'] = bid_cache['u_out'].shift(4)
    df['u_in_lag_back4'] = bid_cache['u_in'].shift(-4)
    df['u_out_lag_back4'] = bid_cache['u_out'].shift(-4)
    df = df.bfill().ffill()
    
    # statistical transform features
    df["u_in_diff_max"] = bid_cache["u_in"].transform("max") - df["u_in"]
    df["u_in_diff_mean"] = bid_cache["u_in"].transform("mean") - df["u_in"]
    df["u_in_deviation"] = ((bid_cache["u_in"].transform("max") - bid_cache["u_in"].transform("min")) - df["u_in"])/bid_cache["u_in"].transform("std")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] = df['u_in_cumsum'] /df['count']
    
    df['time_step_diff'] = bid_cache['time_step'].diff().fillna(0)
    df['ewm_u_in_mean'] = (bid_cache['u_in']\
                           .ewm(halflife=9)\
                           .mean()\
                           .reset_index(level=0,drop=True))
    
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    
    df['time_gap'] = df['time_step'] - df.shift(1).fillna(0)['time_step']
    df['u_in_gap'] = df['u_in'] - df.shift(1).fillna(0)['u_in']
    df['u_in_rate'] = df['u_in_gap'] / df['time_gap']

    df.loc[list(range(0, len(df), 80)), 'time_gap'] = 0
    df.loc[list(range(0, len(df), 80)), 'u_in_gap'] = 0
    df.loc[list(range(0, len(df), 80)), 'u_in_rate'] = 0
    
    df['area_gap'] = df['u_in'] * df['time_gap']
    df['area_cumsum_gap'] = (df['area_gap']).groupby(df['breath_id']).cumsum()
    
    # spectral features
    df['fft_u_in'] = bid_cache['u_in'].transform(ffta)
    df['fft_u_in_w'] = bid_cache['u_in'].transform(fftw)
    df['analytical'] = bid_cache['u_in'].transform(hilbert)
    df['envelope'] = np.abs(df['analytical'])
    df['phase'] = np.angle(df['analytical'])
    df['unwrapped_phase'] = df.groupby(["breath_id"])['phase'].transform(np.unwrap)
    df['phase_shift'] = df.groupby(["breath_id"])['unwrapped_phase'].shift(1).astype(np.float32)
    df['IF'] = df['unwrapped_phase'] - df['phase_shift'].astype(np.float32)
    df = df.fillna(0)
    
    df['exponent'] = (-1 * df['time_step'])/(df['R']*df['C'])
    df['factor'] = df["exponent"].apply(lambda x: np.exp(x))
    df["del_Prc"] = (df['u_in']*df['R'])/df['factor']
    
    df['R__C'] = list(map(lambda x: str(x[0])+str(x[1]), zip(df["R"].values, df["C"].values)))
    df = pd.get_dummies(df)
    return df

In [ ]:
train = add_features(train)
test = add_features(test)

targets = train[['pressure']].to_numpy().reshape(-1, 80)

train.drop(['pressure','id', 'breath_id','one','count', 'exponent', 'analytical'], axis=1, inplace=True)
test.drop(['id', 'breath_id','one','count', 'exponent', 'analytical'], axis=1, inplace=True)

RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [ ]:
def create_model(strategy, data):   
    with strategy.scope():
        inp = L.Input(shape=(data.shape[-2:]),name="Input")

        x1 = L.Bidirectional(L.LSTM(units=1024, return_sequences=True),name="BiLSTM1")(inp)
        x2 = L.Bidirectional(L.LSTM(units=512, return_sequences=True),name="BiLSTM2")(x1)
        x3 = L.Bidirectional(L.LSTM(units=256, return_sequences=True),name="BiLSTM3")(x2)
        x4 = L.Bidirectional(L.LSTM(units=128, return_sequences=True),name="BiLSTM4")(x3)

        z2 = L.Bidirectional(L.GRU(units=256, return_sequences=True),name="BiGRU1")(x2)
        z3 = L.Bidirectional(L.GRU(units=128, return_sequences=True),name="BiGRU2")(L.Add()([x3, z2]))
        z4 = L.Bidirectional(L.GRU(units=128, return_sequences=True),name="BiGRU3")(L.Add()([x4, z3]))

        x = L.Concatenate(axis=2, name="Concat")([x4, z2, z3, z4])

        out = L.Dense(128, activation='selu', kernel_initializer=tf.keras.initializers.lecun_normal(seed=config.random_state))(x)
        out = L.Dense(1)(out)

        model = tf.keras.Model(inputs=inp, outputs=out)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0015), loss='mae')
    
    return model

def plot_hist(hist):
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["val_loss"])
    plt.title("Model Performance")
    plt.ylabel("MAE")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
# Detect hardware, return appropriate distribution strategy
print(tf.version.VERSION)
try:
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.MirroredStrategy() # works on GPU and multi-GPU
    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    tf.config.optimizer.set_jit(True) # XLA compilation
    tf.keras.mixed_precision.experimental.set_policy(policy)
    print('Mixed precision enabled')

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
_ = gc.collect()
kf = KFold(n_splits=config.model_params["NUM_FOLDS"], shuffle=True, random_state=config.random_state)
oof_preds = np.zeros((train.shape[0], 80))
test_preds = []

for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
    if fold >=5:
        print("[INFO] Exiting training loop at fold: ", fold)
        break
    K.clear_session()
    print(f"\nFOLD: {fold}")
    X_train, X_valid = train[train_idx], train[test_idx]
    y_train, y_valid = targets[train_idx], targets[test_idx]
    
    checkpoint_filepath = f"./folds_{fold}.hdf5"
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.75, patience=5, verbose=0)
    es = EarlyStopping(monitor="val_loss", patience=25, verbose=0, mode="min", restore_best_weights=True)
    sv = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_loss',
        verbose=0, 
        save_best_only=True,
        save_weights_only=False,
        mode='auto',
        save_freq='epoch'
    )
    
    model = create_model(strategy, X_train)
    history = model.fit(X_train, y_train,
                        validation_data=(X_valid, y_valid),
                        epochs=config.model_params["EPOCH"],
                        batch_size=config.model_params["BATCH_SIZE"], 
                        callbacks = [lr, es, sv])
    model.load_weights(checkpoint_filepath) 
    
    y_true = y_valid.squeeze().reshape(-1, 1)
    y_pred = model.predict(X_valid, verbose=1, batch_size=config.model_params["BATCH_SIZE"]).squeeze().reshape(-1, 1)
    score = mean_absolute_error(y_true, y_pred)
    print(f"OOF MAE Fold {fold}: {score}")
    oof_preds[test_idx] = y_pred.reshape(-1, 80)
    
    del X_train, X_valid, y_train, y_valid, y_true, y_pred, train_idx, test_idx
    _ = gc.collect()
    
    test_preds.append(model.predict(test, batch_size=config.model_params["BATCH_SIZE"], verbose=1).squeeze().reshape(-1, 1).squeeze())
    plot_hist(history)

print(f"SEED MAE: ", mean_absolute_error(targets.squeeze().reshape(-1, 1), oof_preds.squeeze().reshape(-1, 1)))
pickle.dump(oof_preds, open(f"./oof_preds_res_{config.random_state}.pkl", "wb"))
pickle.dump(test_preds, open(f"./test_preds_res_{config.random_state}.pkl", "wb"))

In [ ]:
submission["pressure"] = np.median(np.vstack(test_preds), axis=0)
submission["pressure"] = np.round((submission.pressure - config.post_processing["min_pressure"])/config.post_processing["diff_pressure"]) * config.post_processing["diff_pressure"] + config.post_processing["min_pressure"]
submission.pressure = np.clip(submission.pressure, config.post_processing["min_pressure"], config.post_processing["max_pressure"])
display(submission.head())
submission.to_csv('submission.csv', index=False)

EOF!